## Library

In [ ]:
!pip install pyLDAvis
!pip install --upgrade pandas==1.2
!pip install textstat
!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
!pip install cartopy

Requirement already up-to-date: pandas==1.2 in /usr/local/lib/python3.7/dist-packages (1.2.0)
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/liba/libarchive/libarchive-dev_3.2.2-3.1ubuntu0.6_amd64.deb  404  Not Found [IP: 91.189.88.152 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
#Loading libraries
# import libarchive
import pydot
import cartopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

#for displaying 500 results in pandas dataframe
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import re
import gensim
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
from collections import defaultdict,Counter
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import string
nltk.download('stopwords')


stop=set(stopwords.words('english'))
plt.style.use('seaborn')


from plotly import tools
import plotly.offline as py
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import textstat
from textblob import TextBlob 
from tqdm import tqdm
from statistics import *
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from gensim import corpora
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##Path

In [ ]:
images_dir = '/content/gdrive/MyDrive/Statistical computing Final project/images'
files_dir = '/content/gdrive/MyDrive/Statistical computing Final project/files'

## Data

In [44]:
from google.colab import drive
drive.mount('/content/gdrive') # 此處需要登入google帳號

Mounted at /content/gdrive


In [ ]:
import pandas as pd
# 獲取授權碼之後輸入即可連動雲端硬碟
train = pd.read_csv("/content/gdrive/MyDrive/Statistical computing Final project/data/Corona_NLP_train.csv",encoding='latin1')
test = pd.read_csv("/content/gdrive/MyDrive/Statistical computing Final project/data/Corona_NLP_test.csv",encoding='latin1')
train['OriginalTweet']=train['OriginalTweet'].astype(str)
train['Sentiment']=train['Sentiment'].astype(str)
test['OriginalTweet']=test['OriginalTweet'].astype(str)
test['Sentiment']=test['Sentiment'].astype(str)
df=pd.read_csv("/content/gdrive/MyDrive/Statistical computing Final project/files/df_new.csv",encoding='latin1')

In [ ]:
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,text,sentiment,length,hash,mentions
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,neutral,111.0,NaN,MeNyrbie Phil_Gahan Chrisitv
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...,positive,237.0,NaN,NaN
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolworths to give elde...,positive,131.0,NaN,NaN
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...,positive,306.0,COVID19france COVID_19 COVID19 coronavirus con...,NaN
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"Me, ready to go at supermarket during the #COV...",negative,310.0,COVID19 coronavirus CoronavirusFrance restezch...,NaN


In [ ]:
#Remove Urls 
def remove_urls(text):
  url_remove = re.compile(r'https?://\S+|www\.\S+')
  return url_remove.sub(r'', text)
df['text_new']=df['text'].apply(str).apply(lambda x:remove_urls(x))
train['text_new']=df['OriginalTweet'].apply(str).apply(lambda x:remove_urls(x))
test['text_new']=df['OriginalTweet'].apply(str).apply(lambda x:remove_urls(x))
#Remove HTML links
def remove_html(text):
  html=re.compile(r'<.*?>')
  return html.sub(r'',text)
df['text']=df['text_new'].apply(str).apply(lambda x:remove_html(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:remove_html(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:remove_html(x))
#Lower text
def lower(text):
  low_text= text.lower()
  return low_text
df['text_new']=df['text'].apply(str).apply(lambda x:lower(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:lower(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:lower(x))
# Number removal
def remove_num(text):
  remove= re.sub(r'\d+', '', text)
  return remove
df['text']=df['text_new'].apply(str).apply(lambda x:remove_num(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:remove_num(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:remove_num(x))
#Remove Punctuations
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))
STOPWORDS = set(stopwords.words('english'))
def punct_remove(text):
  punct = re.sub(r"[^\w\s\d]","", text)
  return punct
df['text_new']=df['text'].apply(lambda x:punct_remove(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:punct_remove(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:punct_remove(x))
# Remove stopwords 
def remove_stopwords(text):
  """custom function to remove the stopwords"""
  return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df['text']=df['text_new'].apply(lambda x:remove_stopwords(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:remove_stopwords(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:remove_stopwords(x))
# Remove mention
def remove_mention(x):
  text=re.sub(r'@\w+','',x)
  return text
df['text_new']=df['text'].apply(lambda x:remove_mention(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:remove_mention(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:remove_mention(x))
# Remove hash
def remove_hash(x):
  text=re.sub(r'#\w+','',x)
  return text
df['text']=df['text_new'].apply(lambda x:remove_hash(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:remove_hash(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:remove_hash(x))
#Remove extra white space left while removing stuff
def remove_space(text):
    space_remove = re.sub(r"\s+"," ",text).strip()
    return space_remove
df['text_new']=df['text'].apply(lambda x:remove_space(x))
train['text_new']=df['text_new'].apply(str).apply(lambda x:remove_space(x))
test['text_new']=df['text_new'].apply(str).apply(lambda x:remove_space(x))
# df = df.drop(columns=['text'])

In [ ]:
text=df['text_new'].to_frame().reset_index()
text['sentiment']=df['sentiment']
text.drop(columns=['index'])
# text.to_csv (r'/content/gdrive/MyDrive/Statistical computing Final project/files/sentiment_text.csv', index = False, header=True)

## Lemmitization

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
lem1=[]
for i in range(len(df['text_new'])):
  temp = [wnl.lemmatize(word) for word in df['text_new'][i].split()]
  tem=" ".join(temp)
  lem1+=[tem]
lem1

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


['menyrbie phil_gahan chrisitv',
 'advice talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med order',
 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid outbreak',
 'food stock one empty please dont panic enough food everyone take need stay calm stay safe covidfrance covid_ covid coronavirus confinement confinementotal confinementgeneral',
 'ready go supermarket covid outbreak im paranoid food stock litteraly empty coronavirus serious thing please dont panic cause shortage coronavirusfrance restezchezvous stayathome confinement',
 'news regionãâs first confirmed covid case came sullivan county last week people flocked area store purchase cleaning supply hand sanitizer food toilet paper good tim_dodson report',
 'cashier grocery store sharing insight covid_ prove credibility commented im civics class know im talking',
 'supermar

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
lem_train=[]
for i in range(len(train['text_new'])):
  temp = [wnl.lemmatize(word) for word in train['text_new'][i].split()]
  tem=" ".join(temp)
  lem_train+=[tem]
lem_train

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['menyrbie phil_gahan chrisitv',
 'advice talk neighbour family exchange phone number create contact list phone number neighbour school employer chemist gp set online shopping account po adequate supply regular med order',
 'coronavirus australia woolworth give elderly disabled dedicated shopping hour amid covid outbreak',
 'food stock one empty please dont panic enough food everyone take need stay calm stay safe covidfrance covid_ covid coronavirus confinement confinementotal confinementgeneral',
 'ready go supermarket covid outbreak im paranoid food stock litteraly empty coronavirus serious thing please dont panic cause shortage coronavirusfrance restezchezvous stayathome confinement',
 'news regionãâs first confirmed covid case came sullivan county last week people flocked area store purchase cleaning supply hand sanitizer food toilet paper good tim_dodson report',
 'cashier grocery store sharing insight covid_ prove credibility commented im civics class know im talking',
 'supermar

In [ ]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
lem_test=[]
for i in range(len(test['text_new'])):
  temp = [wnl.lemmatize(word) for word in test['text_new'][i].split()]
  tem=" ".join(temp)
  lem_test+=[tem]
lem_test

In [ ]:
from pandas import DataFrame
df1 = DataFrame (lem1,columns=['lemma_text'])
df1['sentiment']=df['sentiment']
df1.head()
df1.to_csv (r'/content/gdrive/MyDrive/Statistical computing Final project/files/lemma_text.csv', index = False, header=True)

In [ ]:
from pandas import DataFrame
l_train = DataFrame (lem_train,columns=['lemma_text'])
l_train['sentiment']=df['sentiment']
l_train.head()
l_train.to_csv (r'/content/gdrive/MyDrive/Statistical computing Final project/files/lemma_train.csv', index = False, header=True)

In [ ]:
from pandas import DataFrame
l_test = DataFrame (lem_test,columns=['lemma_text'])
l_test['sentiment']=df['sentiment']
l_test.head()
l_test.to_csv (r'/content/gdrive/MyDrive/Statistical computing Final project/files/lemma_test.csv', index = False, header=True)

In [ ]:
nltk.download('punkt')
tokens=[]
for i in range(len(lem1)):
  tokens.append(nltk.word_tokenize(lem1[i]))
tokens

In [ ]:
# import numpy as np
# tokens_all=np.array(tokens)
# np.save("/content/gdrive/MyDrive/Statistical computing Final project/files/tokens_all.npy",tokens_all) # 保存为.npy格式

## Stemming

In [ ]:
# from nltk import LancasterStemmer
# ls = LancasterStemmer()
# token_lsstem1=[]
# for i in range(len(df['text_new'])):
#   temp = [ls.stem(word) for word in df['text_new'][i].split()]
#   tem=" ".join(temp)
#   token_lsstem1+=[tem]
# token_lsstem1